In [ ]:
%matplotlib notebook


# DS-CNN/KWS inference

This tutorial illustrates how to build a basic speech recognition
Akida network that recognizes thirty-two different words.

The model will be first defined as a CNN and trained in Keras, then quantized using [QuantizeML](../../user_guide/quantizeml.html)_. and converted using [CNN2SNN](../../user_guide/cnn2snn.html)_.

This example uses a Keyword Spotting Dataset prepared using **TensorFlow** [audio recognition
example](https://www.tensorflow.org/tutorials/audio/simple_audio)_ utils.

The words to recognize have been converted to [spectrogram images](https://github.com/tensorflow/docs/blob/master/site/en/r1/tutorials/sequences/audio_recognition.md#how-does-this-model-work)_
that allows us to use a model architecture that is typically used for image recognition tasks.


## 1. Load the preprocessed dataset

The TensorFlow [speech_commands](https://www.tensorflow.org/datasets/catalog/speech_commands)_
dataset is used for training and validation. All keywords except "backward",
"follow" and "forward", are retrieved. These three words are kept to
illustrate the edge learning in this
[edge example](../edge/plot_1_edge_learning_kws.html)_.
The raw audio data are not directly used for training. As noted above, they have been
preprocessed, transforming the audio files into MFCC features, well-suited for CNN networks.
A pickle file containing the preprocessed data is available on our data server.




In [ ]:
import pickle

from tensorflow.keras.utils import get_file

# Fetch pre-processed data for 32 keywords
fname = get_file(
    fname='kws_preprocessed_all_words_except_backward_follow_forward.pkl',
    origin="https://data.brainchip.com/dataset-mirror/kws/kws_preprocessed_all_words_except_backward_follow_forward.pkl",
    cache_subdir='datasets/kws')
with open(fname, 'rb') as f:
    [_, _, x_valid, y_valid, _, _, word_to_index, _] = pickle.load(f)

# Preprocessed dataset parameters
num_classes = len(word_to_index)

print("Wanted words and labels:\n", word_to_index)

## 2. Load a pre-trained native Keras model

The model consists of:

* a first convolutional layer accepting dense inputs (images),
* several separable convolutional layers preserving spatial dimensions,
* a global pooling reducing the spatial dimensions to a single pixel,
* a final dense layer to classify words.

All layers are followed by a batch normalization and a ReLU activation.




In [ ]:
from tensorflow.keras.models import load_model

# Retrieve the model file from the BrainChip data server
model_file = get_file("ds_cnn_kws.h5",
                      "https://data.brainchip.com/models/AkidaV2/ds_cnn/ds_cnn_kws.h5",
                      cache_subdir='models')

# Load the native Keras pre-trained model
model_keras = load_model(model_file)
model_keras.summary()

In [ ]:
import numpy as np

from sklearn.metrics import accuracy_score

# Check Keras Model performance
potentials_keras = model_keras.predict(x_valid)
preds_keras = np.squeeze(np.argmax(potentials_keras, 1))

accuracy = accuracy_score(y_valid, preds_keras)
print("Accuracy: " + "{0:.2f}".format(100 * accuracy) + "%")

## 3. Load a pre-trained quantized Keras model

The above native Keras model is quantized and fine-tuned (QAT) to recover some accuracy. The first
convolutional layer uses 8-bit weights, but other layers use 4-bit weights, activations are all
4-bit.




In [ ]:
from akida_models import ds_cnn_kws_pretrained

# Load the pre-trained quantized model
model_keras_quantized = ds_cnn_kws_pretrained()
model_keras_quantized.summary()

# Check Model performance
potentials_keras_q = model_keras_quantized.predict(x_valid)
preds_keras_q = np.squeeze(np.argmax(potentials_keras_q, 1))

accuracy_q = accuracy_score(y_valid, preds_keras_q)
print("Accuracy: " + "{0:.2f}".format(100 * accuracy_q) + "%")

## 4. Conversion to Akida

We convert the model to Akida and then evaluate the performance on the dataset.




In [ ]:
from cnn2snn import convert

# Convert the model
model_akida = convert(model_keras_quantized)
model_akida.summary()

In [ ]:
# Check Akida model performance
preds_akida = model_akida.predict_classes(x_valid, num_classes=num_classes)

accuracy = accuracy_score(y_valid, preds_akida)
print("Accuracy: " + "{0:.2f}".format(100 * accuracy) + "%")

# For non-regression purposes
assert accuracy > 0.9

## 5. Confusion matrix

The confusion matrix provides a good summary of what mistakes the
network is making.

Per scikit-learn convention it displays the true class in each row (ie
on each row you can see what the network predicted for the corresponding
word).

Please refer to the Tensorflow [audio
recognition](https://github.com/tensorflow/docs/blob/master/site/en/r1/tutorials/sequences/audio_recognition.md#confusion-matrix)_
example for a detailed explanation of the confusion matrix.




In [ ]:
import itertools
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

# Create confusion matrix
cm = confusion_matrix(y_valid, preds_akida,
                      labels=list(word_to_index.values()))

# Normalize
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Display confusion matrix
plt.rcParams["figure.figsize"] = (16, 16)
plt.figure()

title = 'Confusion matrix'
cmap = plt.cm.Blues

plt.imshow(cm, interpolation='nearest', cmap=cmap)
plt.title(title)
plt.colorbar()
tick_marks = np.arange(len(word_to_index))
plt.xticks(tick_marks, word_to_index, rotation=45)
plt.yticks(tick_marks, word_to_index)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j,
             i,
             format(cm[i, j], '.2f'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.autoscale()
plt.show()